In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.animation as animation
import glob
import re


path_1MJ_YESfeel_lower = "/home/matias/Simulations/mi_fargo3d/outputs/flyby2d_1MJ_NOfeel_lower/"
path = path_1MJ_YESfeel_lower

# Identificar el número de outputs disponibles
# Patrón para archivos gasdens*.dat con un número entero
pattern = re.compile(r'gasdens(\d+)\.dat')
# Lista de archivos que coinciden con el patrón gasdens*.dat
files = glob.glob(path + "gasdens*.dat")
# Filtrar archivos que se ajustan al patrón correcto
valid_files = [f for f in files if pattern.match(os.path.basename(f))]
# Contar el número de archivos válidos
Ntot = len(valid_files)
print("En total hay: ", Ntot, "outputs")

#UNITS
gamma=1.6666
mu=2.35 #
kb=1.380650424e-16 #ergs/K
R_gas=8.314472e7 #erg/K/mol 
G = 6.67259e-8 #cm**3 g**-1 s**-2
mp= 1.672623099e-24 # g

unit_mass   = 1.9891e33 # g
unit_length = 1.49598e13 #cm
unit_density=(unit_mass)/(unit_length**3) # g/cm**3
unit_surf_density = (unit_mass/unit_length**2) # g/cm**2
unit_time     = np.sqrt( pow(unit_length,3.) / G / unit_mass) #s 
unit_energy   = unit_mass/(unit_time)**2/(unit_length)  #erg/cm3 = gr/s2/cm  #juan garrido
unit_surf_energy = unit_energy / unit_length
unit_velocity = unit_length / unit_time  # cm/s

unit_temperature  = ((G*mp*mu)/(kb))*(unit_mass/(unit_length)) #K


variables_par = np.genfromtxt(path+"/variables.par",dtype={'names': ("parametros","valores"),'formats': ("|S30","|S300")}).tolist()#Lee archivo var    iable.pary convierte a string       esita un int o float
parametros_par, valores_par = [],[]                                                                                                                                                                                                                         #Reparte entre parametros y valores
for posicion in variables_par:                                                                                                                                                                                                                              #
        parametros_par.append(posicion[0].decode("utf-8"))                                                                                                                                                                                  #
        valores_par.append(posicion[1].decode("utf-8"))                                                                                                                                                                                     #

def P(parametro):
        return valores_par[parametros_par.index(parametro)] 



def compute_accretion_rate(output_number, path, r0):
    """
    Calcula la tasa de acreción en el radio r0 para un output_number dado.

    Parámetros:
    - output_number: número del output (snapshot) de la simulación.
    - path: ruta al directorio donde están los datos de la simulación.
    - r0: radio seleccionado donde calcular la tasa de acreción (en unidades de AU).

    Retorna:
    - accretion_rate: tasa de acreción en unidades de masas solares por año (Msun/yr).
    """

    # Cargar los datos de densidad superficial (gasdens) y velocidad radial (gasvy)
    dens_data = np.fromfile(path + f"gasdens{output_number}.dat").reshape(NY, NX) * unit_surf_density  # [g/cm^2]
    vr_data = np.fromfile(path + f"gasvy{output_number}.dat").reshape(NY, NX) * unit_velocity  # [cm/s]

    # Obtener el radio y ángulo azimutal en cada celda
    Phi = 0.5 * (domain_x[1:] + domain_x[:-1])  # [rad]
    R = 0.5 * (domain_y[1:] + domain_y[:-1])    # [AU]

    # Si la malla es uniforme en phi
    Delta_phi = 2 * np.pi / NX  # [rad]

    # Encontrar el índice radial más cercano a r0
    radial_index = np.argmin(np.abs(R - r0))
    r_selected = R[radial_index] * unit_length  # Convertir a [cm]

    # Extraer densidad y velocidad radial en el radio seleccionado
    Sigma_r = dens_data[radial_index, :]  # [g/cm^2]
    vr_r = vr_data[radial_index, :]       # [cm/s]

    # Calcular el flujo de masa en cada celda azimutal
    mass_flux = -Sigma_r * vr_r * r_selected * Delta_phi  # [g/s]

    # Sumar sobre todas las celdas azimutales para obtener la tasa de acreción total
    accretion_rate = np.sum(mass_flux)  # [g/s]

    # Convertir la tasa de acreción a unidades de masas solares por año
    accretion_rate_msun_per_year = accretion_rate * (3.154e7) / 1.989e33  # [Msun/yr]

    return accretion_rate_msun_per_year

